# Mass Balances for Block Flow Diagrams

## Separating Sugar from Salt

In [38]:
import pyomo.environ as pyo

# concrete model
bfd = pyo.ConcreteModel("Block Flow Diagram")

# components and streams sets
bfd.components = pyo.Set(initialize=("Sucrose", "NaCl", "Ethanol"))
bfd.streams = pyo.Set(initialize=("Feed", "Recycle", "Blend", "Decant", "Solids", "Product"))

# stream table
bfd.f = pyo.Var(bfd.components, bfd.streams, domain=pyo.NonNegativeReals)

# unit mass balances
def mass_balance(inputs, outputs):
    return pyo.Constraint(bfd.components, rule=lambda m, c:
                sum(bfd.f[c, s] for s in inputs) == sum(bfd.f[c, s] for s in outputs))

# feed specification
bfd.feed = pyo.ConstraintList()
bfd.feed.add(bfd.f["Sucrose", "Feed"] == 10)
bfd.feed.add(bfd.f["NaCl", "Feed"] == 10)

# recycle specifications
bfd.recycle_specs = pyo.ConstraintList()
#bfd.recycle_specs.add(bfd.f["Ethanol", "Recycle"] <= 1000)

# mixer specifications
bfd.mixer = mass_balance(["Feed", "Recycle"], ["Blend"])
    
# settler specifications
bfd.settler = mass_balance(["Blend"], ["Decant", "Solids"])
bfd.settler_specs = pyo.ConstraintList()
bfd.settler_specs.add(bfd.f["Ethanol", "Solids"] == 0)
bfd.settler_specs.add(bfd.f["NaCl", "Decant"] <= 0.00065*bfd.f["Ethanol", "Decant"])
bfd.settler_specs.add(bfd.f["Sucrose", "Decant"] <= 0.0047*bfd.f["Ethanol", "Decant"])

# evaporator specifications
bfd.evaporator = mass_balance(["Decant"], ["Recycle", "Product"])
bfd.evaporator_specs = pyo.ConstraintList()
bfd.evaporator_specs.add(bfd.f["Ethanol", "Product"] == 0)
bfd.evaporator_specs.add(bfd.f["NaCl", "Recycle"] == 0)
bfd.evaporator_specs.add(bfd.f["Sucrose", "Recycle"] == 0)

# objective
bfd.obj = pyo.Objective(expr = bfd.f["Sucrose", "Product"], sense=pyo.maximize)

solver = pyo.SolverFactory('cbc')
solver.solve(bfd)

# extract solution
import pandas as pd
pd.DataFrame({s: {c: bfd.f[c, s]() for c in bfd.components} for s in bfd.streams})

,Feed,Recycle,Blend,Decant,Solids,Product
Sucrose,10.0,0.0000,10.0000,10.0000,0.0,10.0
NaCl,10.0,0.0000,10.0000,0.0000,10.0,0.0
Ethanol,0.0,2127.6596,2127.6596,2127.6596,0.0,0.0


![](https://processdesign.mccormick.northwestern.edu/images/5/54/Benzene_prod_example.JPG)

In [35]:
import pyomo.environ as pyo

# molecular weights
mol_weights = {"hydrogen": 2.0, "methane": 16.04, "benzene": 78.11, "toluene" : 92.14}

# reactions
reactions = {"Conversion of Toluene": {"toluene": -1, "hydrogen": -1, "benzene": 1, "methane": 1}}


bfd = pyo.ConcreteModel()

# components and streams sets
bfd.components = pyo.Set(initialize=mol_weights.keys())
bfd.streams = pyo.Set(initialize=("Toluene Feed", "Hydrogen Feed", "Effluent", "Mixed Gas", 
                                  "Benzene", "Mixed Liquids", "Toluene Recycle"))
bfd.reactions = pyo.Set(initialize=reactions.keys())

# stream table
bfd.f = pyo.Var(bfd.components, bfd.streams, domain=pyo.NonNegativeReals)

# extent of reaction

# unit mass balances
def mass_balance(inputs, outputs):
    return pyo.Constraint(bfd.components, rule=lambda m, c: 
                sum(bfd.f[c, s] for s in inputs) == sum(bfd.f[c, s] for s in outputs))




In [62]:
# process units
def mixer_rule(b, n):
    b.IN = pyo.RangeSet(n)
    b.inflow = pyo.Var(b.IN, bfd.Species, domain=pyo.NonNegativeReals)
    b.outflow = pyo.Var(bfd.Species, domain=pyo.NonNegativeReals)
    b.mass_bal = pyo.Constraint(bfd.Species, rule = lambda b, s:
            0 == sum(b.inflow[n, s] for n in b.IN) - b.outflow[s])
    
bfd.mixer = pyo.Block(rule=lambda b: mixer_rule(b, 2))



def settler_rule(b):
    pass
bfd.settler = pyo.Block(rule=settler_rule)

def evaporator_rule(b):
    b.inflow = pyo.Var(bfd.Species, domain=pyo.NonNegativeReals)
bfd.evaporator = pyo.Block(rule=evaporator_rule)

solver = pyo.SolverFactory('cbc')
#solver.solve(bfd)





bfd.pprint()

4 Set Declarations
    Species : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'Sucrose', 'NaCl', 'Ethanol'}
    Streams : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'Feed', 'Recycle', 'Product'}
    feed_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    q_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain          : Size : Members
        None :     2 : Streams*Species :    9 : {('Feed', 'Sucrose'), ('Feed', 'NaCl'), ('Feed', 'Ethanol'), ('Recycle', 'Sucrose'), ('Recycle', 'NaCl'), ('Recycle', 'Ethanol'), ('Product', 'Sucrose'), ('Product', 'NaCl'), ('Product', 'Ethanol')}

1 Var Declarations
    q : Size=9, Index=q_index
        Key                    : Lower : Value : Upper : Fixed : Stale : Domain
           (